In [8]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from utils import dasker
import pandas as pd
import dask

In [10]:
import dask.dataframe as dd

In [11]:
from dask.distributed import Client
from dask_ml.datasets import make_classification
from dask_ml.model_selection import train_test_split

In [12]:
client = dasker.get_global_client()

connected to cluster dev.7a3159da4bca48e49a206d616133ca78


In [13]:
def get_covid_dataset():
    df = pd.read_csv('/home/wasif/python-asd/xai/app/test/data/20220319_covid_merge_processed.csv', sep=",")
    X = df[df.columns[df.columns!='y']]
    # X = X.drop(DROP_LIST, axis = 1)
    y = df[df.columns[df.columns=='y']]
    return X, y

In [14]:
X, y = get_covid_dataset()

In [15]:
# X.dtype

In [16]:
# X = dd.from_pandas(X, npartitions=3)
# y = dd.from_pandas(y, npartitions=3)


In [17]:
import dask.array as da

x = da.from_array(X.values)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
X_train_da = da.from_array(X_train.values, chunks=(1000, 1000))
y_train_da = da.from_array(y_train.values, chunks=(1000, 1000))

In [20]:
y_train_da

dask.array<array, shape=(171, 1), dtype=float64, chunksize=(171, 1), chunktype=numpy.ndarray>

In [22]:
from scipy import stats # import uniform, loguniform
import xgboost as xgb



params_fixed = {'objective'   : 'count:poisson', 
                'random_state': 432,
                'n_jobs'      : 1
               }

params_hyp = {
    'max_depth'        : stats.randint(3,6),
    'learning_rate'    : stats.loguniform(0.01, 1.0),
    'n_estimators'     : stats.randint(100, 1000),
    'subsample'        : [0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha'        : stats.loguniform(0.01, 1.0),
    'reg_lambda'       : stats.loguniform(0.01, 1.0),
    'scale_pos_weight' : [2,3,4]
    }

# model = XGBRegressor(**params_fixed)

model = xgb.dask.DaskXGBRegressor(**params_fixed)


In [50]:
from scipy import stats # import uniform, loguniform
import lightgbm as lgb



params_fixed = {'objective'   : 'count:poisson', 
                'random_state': 432,
                'n_jobs'      : 1
               }

params_hyp = {
    'max_depth'        : stats.randint(3,6),
    'learning_rate'    : stats.loguniform(0.01, 1.0),
    'n_estimators'     : stats.randint(100, 1000),
    'subsample'        : [0.6, 0.7, 0.8, 0.9, 1.0],
    'reg_alpha'        : stats.loguniform(0.01, 1.0),
    'reg_lambda'       : stats.loguniform(0.01, 1.0),
    'scale_pos_weight' : [2,3,4]
    }

model = lgb.DaskLGBMRegressor(**params_fixed)

In [51]:


# clf = XGBRegressor(n_estimators=1000, max_depth=7)


In [23]:
n_examples = 20 * len(X_train_da)  # 20 passes through dataset for best model

n_params = 100  # sample approximately 100 parameters; more than 94 will be sampled

In [24]:
from dask_ml.model_selection import HyperbandSearchCV

search = HyperbandSearchCV(model, params_hyp, max_iter=n_examples, random_state=0)

# Perform Hyperparameter Tuning
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=SEED)



search.fit(X_train_da, y_train_da)

AttributeError: 'DaskXGBRegressor' object has no attribute 'partial_fit'

In [ ]:
from dask_ml.model_selection import HyperbandSearchCV, RandomizedSearchCV

search = RandomizedSearchCV(model, params_hyp, random_state=432)

# Perform Hyperparameter Tuning
#cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=SEED)



search.fit(X_train_da, y_train_da)

('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 7, 0) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 7, 2) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 7, 1) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 8, 2) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 8, 0) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 9, 0) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 8, 1) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 7, 0) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 7, 2) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 7, 1) has failed... retrying
('daskxgbregressor-fit-score-4222598ddb2d038dd032f76a4fa54fe3', 8, 0) 

In [ ]:
search.best_score_